In [2]:
'''Code to train Autoencoder for fingerprints'''
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D,LeakyReLU
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint

import matplotlib.pyplot as plt

In [3]:
data_pd = pd.read_pickle('/home/cc/Data/Zips/cm_index_df_fixed.p')

In [ ]:
data_pd.shape

In [4]:
data_pd['FilePath']= data_pd.FilePath.apply(lambda x: os.path.join('/home/cc/Data/Images',x))

In [ ]:
data_live = data_pd[data_pd.live_spoof == 0] # only live samples

In [ ]:
data_live.shape

In [ ]:
data_live['Collection'] = data_live.CaptureID.apply(lambda x: x[0:3])

In [ ]:
data_live['Collection'].unique()

In [ ]:
resize_df = data_live[data_live.Collection =='Pre']
resize_df.shape

In [ ]:
def rescaleimage(image):
    image=(image-127.5)/127.5
    return image
def reverseimage(image):
    image=(image/2)+0.5
    return image

In [ ]:
img_rows = 512
img_cols = 512
channels = 3 # I am using channel last models
img_shape = (img_rows, img_cols, channels)
latent_dim = 512
batch_size=256

inputTrain ='/home/cc/Data/Images'

datagen=ImageDataGenerator(preprocessing_function=rescaleimage)
#datagen=ImageDataGenerator(preprocessing_function=rescaleimage, horizontal_flip=True, vertical_flip=True)
batch_size=256
image_generator=datagen.flow_from_dataframe(
    directory = inputTrain,
    dataframe = resize_df, 
    color_mode='rgb',
    x_col = 'FilePath',
    batch_size= batch_size,
    target_size=(img_rows,img_cols),
    class_mode='input',
    shuffle=True
    )

In [5]:
initialize=RandomNormal(mean=0.0, stddev=0.02, seed=None)
'''
G_ENC
'''
CAE = Sequential()
CAE.add(Conv2D(16, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, input_shape=(512,512,3), padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(32, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(64, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(128, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(256, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(512, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(1024, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Flatten())
CAE.add(Dense(512))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

'''
G-DEC
'''
CAE.add(Dense(1024 * 4 * 4))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Reshape((4, 4, 1024)))
CAE.add(Conv2DTranspose(512, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))        
CAE.add(Conv2DTranspose(256, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(128, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))        
CAE.add(Conv2DTranspose(64, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(32, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(16, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(3, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('tanh'))

optimizer=Adam(lr=0.0001, beta_1=0.5, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=True)
CAE.compile(loss='mse', optimizer=optimizer)

In [6]:
latest = tf.train.latest_checkpoint('/home/cc/Data/CAE')

In [7]:
CAE.load_weights(latest)

In [8]:
CAE.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      784       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 16)      64        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      8224      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        3

In [9]:
CAE.save_weights('/home/cc/Data/CAE/CAE_v2_weights.h5', 'h5')

In [10]:
CAE.save('/home/cc/Data/CAE/CAE_v2.h5', 'h5')

In [ ]:
x1 = image_generator.next()[0]
x = CAE.predict(x1)

In [ ]:
img1 = []
img2 = []
N = [0,1, 2, 3, 4, 5]
for n in N:
    img1.append(x1[n,:,:,:])
    img2.append(x[n,:,:,:])

In [ ]:
for n in N:
    img1[n] = img1[n]/2
    img1[n] = img1[n]+0.5
    img2[n] = img2[n]/2
    img2[n] = img2[n]+0.5

In [ ]:
import matplotlib.pyplot as plt
i=1
plt.figure(figsize=(10, 10))
for n in N:
    plt.subplot(6,2,i)
    plt.imshow(img1[n])
    i= i+1
    plt.subplot(6,2,i)
    plt.imshow(img2[n])
    i=i+1

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img1[1], cmap='gray')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img2[1], cmap='gray')